<a href="https://colab.research.google.com/github/shhdwi/circuit/blob/main/Circuit_LambdaFunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain
!pip install llama-index
!pip install supabase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.5/493.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.8 MB/s eta 0:00:00
  Attempting uninstall: typing-inspect
    Found existing installation: typing-inspect 0.9.0
    Uninstalling typing-inspect-0.9.0:
      Successfully uninstalled typing-inspect-0.9.0
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.10
    Uninstalling SQLAlchemy-2.0.10:
      Successfully uninstalled SQLAlchemy-2.0.10
  Attempting uninstall: fsspec
    Found existi

In [3]:
import os
import getpass
from supabase.client import Client, create_client

In [4]:
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [13]:
os.environ['SUPABASE_URL'] = getpass.getpass('Supabase URL:')

Supabase URL:··········


In [6]:
os.environ['SUPABASE_SERVICE_KEY'] = getpass.getpass('Supabase Service Key:')

Supabase Service Key:··········


In [8]:
APIFY_API_KEY = getpass.getpass('Apify key')

Apify key··········


In [14]:
import os
from supabase.client import Client, create_client

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

In [23]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import SupabaseVectorStore
from langchain.document_loaders import TextLoader
from langchain.document_loaders.base import Document
from langchain.indexes import VectorstoreIndexCreator
from langchain.utilities import ApifyWrapper

In [40]:
# def tranform_dataset_item(item):
#     return Document(
#         page_content=item.get("description"),
#         metadata={
#             "title": item.get("title"),
#             "url": item.get("url")
#         },
#     )

# ApifyActor = download_loader("ApifyActor")

# reader = ApifyActor(APIFY_API_KEY)
# documents = reader.load_data(
#     actor_id="pocesar/shopify-scraper",
#     run_input={"startUrls": [{"url": "https://www.boat-lifestyle.com"}]},
#     dataset_mapping_function=tranform_dataset_item,
# )
os.environ["APIFY_API_TOKEN"]= APIFY_API_KEY
apify = ApifyWrapper()

loader = apify.call_actor(
    actor_id="pocesar/shopify-scraper",
    run_input={"startUrls": [{"url": "https://www.boat-lifestyle.com"}]},
    dataset_mapping_function=lambda item: Document(
        page_content=(item["title"]+ "url: " +item["url"]+ "description: " +item["description"]) or "", source=item["description"], url=item["url"]
    ),
)
docs = loader.load()

In [41]:
embeddings = OpenAIEmbeddings()
vector_store = SupabaseVectorStore.from_documents(
    docs, embeddings, client=supabase
)